Import the libraries

In [ ]:
import pandas as pd
import numpy as np
import statistics
import matplotlib.pyplot as plt

Load thedataframe

In [ ]:
#load the dataframe
df = pd.read_csv('C:\\Games\\DB project\\Shai_Ai_Task\\Salaries.csv') #dont forget to change the path of the dataframe

1.Explore the dataframe

In [ ]:
print(df.info())  #show information about the dataframe

In [ ]:
print("Shape of DataFrame: ",df.shape) #show number  of rows and columns in the dataframe

2.Cleaning the dataframe

In [ ]:
print(df.dtypes) #show the datatype of the  columns in the dataframe

In [ ]:
print(df.isnull().any())  #check if there  is any missing data

In [ ]:
print(df.isnull().sum()) #show number of messing values in each col in the dataframe

In [ ]:
df.fillna(0, inplace=True) # replace the missing values with zero

In [ ]:
print(df.isnull().any())  

In [ ]:
# check if there is a negative values in the numerical cols in the dataframe
for col in df.select_dtypes(include=[np.number]).columns: 
    if (df[col] < 0).any():
        print(f"Column '{col} :  T")
    else:
        print(f"Column '{col} :  F")

In [ ]:
# replace the negative values in the with zero
for col in df.select_dtypes(include=[np.number]).columns: 
    df[col] = df[col].clip(lower=0)

In [ ]:
# check again if there is a negative values in the dataframe
for col in df.select_dtypes(include=[np.number]).columns:
    if (df[col] < 0).any():
        print(f"Column '{col} :  T")
    else:
        print(f"Column '{col} :  F")

In [ ]:
# drop the unnecessary cols in the dataframe
df = df.drop('Notes', axis=1)
df = df.drop('Status', axis=1)
df = df.drop('EmployeeName',axis=1)
df = df.drop('Agency',axis=1)
df=df.drop('Id',axis=1)

In [ ]:
print(df.info())

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist() # store the numeric cols in temperory dataframe


In [ ]:
# calculate the outliers in the numeric dataframe
Q1=df[numeric_cols].quantile(.25)
Q3=df[numeric_cols].quantile(.75)
IQR=Q3-Q1
outliers = (df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))
print(outliers.sum())

In [ ]:
print(df.duplicated().sum()) # show the number of duplicated rows in the dataframe

In [ ]:
df.drop_duplicates(inplace=True) #remove the duplicated rows in the daraframe

In [ ]:
print(df.duplicated().sum())

In [ ]:
#checks if the numeric cols in the dataframe is right_calculated
mask1 = df['TotalPay'] == df['BasePay'] + df['OvertimePay'] + df['OtherPay']
if mask1.all:
    print("[TotalPay] is right_calculated\n ")
else:
    print("[TotalPay] isnt right_calculated\n ")
mask2 = ((df['TotalPayBenefits'] == df['TotalPay'] + df['Benefits']) & (df['TotalPayBenefits'] == df['BasePay'] + df['OvertimePay'] + df['OtherPay'] + df['Benefits']))
if mask2.all:
    print("[TotalPayBenefits] is right_calculated\n")
else:
    print("[TotalPayBenefits] isnt right_calculated\n")

3.Statistics

In [ ]:
stats = df['TotalPayBenefits'].agg(['mean', 'median', statistics.mode, 'min', 'max', 'std', lambda x: x.max() - x.min()]) # Calculate the basic statistics of the salary
print(stats)

In [ ]:
# Shows the mode you choose and the modes before it
def modes(C):
    values, counts = np.unique(df['TotalPayBenefits'], return_counts=True)
    sorted_indices = np.argsort(-counts)
    modes = []
    for i in range(C):
        if len(values) > i:
            Mode = values[sorted_indices[i]]
            Mode_Count = counts[sorted_indices[i]]
            modes.append((Mode, Mode_Count))
            print(f"Mode number {i+1} is  : {Mode} , Count: {Mode_Count}\n")
        else:
            print(f"There is no mode number {i+1}\n")
            break
modes(2)

4.Visualizing

In [ ]:
# Histogram shows the freq of the Salaries
plt.hist(df['TotalPayBenefits'], bins=11, edgecolor='black')
plt.title('Salary Distribution')
plt.xlabel('Salary')
plt.ylabel('Frequency')

In [ ]:
# A piechart to show the percnetage of employees with the same job 
sample_df = df.sample(n=383)
ss = sample_df['JobTitle'].value_counts()
ss = ss[ss > 5]
print(ss)
P1 = plt.pie(ss, labels=ss.index, autopct='%1.1f%%')
plt.title('Employees by Job_Title')

In [ ]:
sub_df = df[df['JobTitle'].str.contains('Department', case=False)].copy() #create a sub_dataframe with  rows containing 'Department' in JobTitle

In [ ]:
# A piechart shows the percnetage of the employees under the same Department
sd = sub_df['JobTitle'].value_counts()
sub_df.loc[:, 'Department'] = np.where(sub_df['JobTitle'].str.contains('Fire Department', case=False), 'Fire Department',
np.where(sub_df['JobTitle'].str.contains('Police Department', case=False), 'Police Department', 'Other'))
sub_df = sub_df[~(sub_df['Department'] == 'Other')]
z = sub_df['Department'].value_counts()
print(z)
plt.pie(z, labels=z.index, autopct='%1.1f%%')
plt.title('Employees by Departments')

5.Analying the dataframe

In [ ]:
sm = df.groupby('Year')['TotalPayBenefits'].agg(['mean', 'median', 'std'])# Calculates the summary statistics of the salaries grouped by the year
print(sm)

In [ ]:
smz = sub_df.groupby('Department')['TotalPayBenefits'].agg(['mean','median','std'])  # Calculates the summary statistics of the salaries grouped by the Department
print(smz)


In [ ]:
sdf = sub_df.groupby(['Department','Year'])['TotalPayBenefits'].agg(['mean','median','std']) # Calculates the summary statistics of the salaries grouped by the (Departmebts_Year)
print(sdf)

In [ ]:
# Compares between the previous means values
mean1 = df.groupby('Year')['TotalPayBenefits'].mean()
mean2 = sub_df.groupby('Department')['TotalPayBenefits'].mean()
mean3 = sub_df.groupby('Year')['TotalPayBenefits'].mean()
mean4 = sub_df.groupby(['Department','Year'])['TotalPayBenefits'].mean()
df_mean1 = mean1.reset_index().rename(columns={'TotalPayBenefits': 'mean1'})
df_mean2 = mean2.reset_index().rename(columns={'TotalPayBenefits': 'mean2'})
df_mean3 = mean3.reset_index().rename(columns={'TotalPayBenefits': 'mean3'})
df_mean4 = mean4.reset_index().rename(columns={'TotalPayBenefits': 'mean4'})
df_means = pd.merge(df_mean1, df_mean4, on='Year', how='outer')
df_means = pd.merge(df_means, df_mean2, on='Department', how='outer')
df_means = pd.merge(df_means, df_mean3, on='Year', how='outer')
print(df_means)

In [ ]:
mean1.plot(kind='bar')
plt.title('Average_Salaries per year')
plt.ylabel('Average_Salaries')
plt.xticks(rotation=0)

In [ ]:
mean2.plot(kind='bar')
plt.title('Average_Salaries per Departments')
plt.ylabel('Average_Salaries')
plt.xticks(rotation=0)

In [ ]:
mean3.plot(kind='bar')
plt.title('Mean Average_Salaries per Year for Departments')
plt.ylabel('Average_Salaries')
plt.xticks(rotation=0)

In [ ]:
a = mean4.plot(kind='bar')
labels = ['FD_2011', 'FD_2012', 'FD_2013', 'FD_2014', 'PD_2011', 'PD_2012', 'PD_2013', 'PD_2014']
a.set_xticklabels(labels)
plt.title('Average_Salaries per Departmebts_Year')
plt.ylabel('Average_Salaries')
plt.xticks(rotation=0)

6.Correlations

In [ ]:
#  calculate_and_visualize_correlation
for col in numeric_cols:
        if col != 'TotalPayBenefits': 
            correlation = df['TotalPayBenefits'].corr(df[col])
            print(f'Correlation between Salary and {col}: {correlation}')
            plt.scatter(df['TotalPayBenefits'], df[col])				
            plt.xlabel('Salary')
            plt.ylabel(col)
            plt.title(f'Salary vs {col}')
            plt.show()